<a href="https://colab.research.google.com/github/lsmanoel/PowerEletronics/blob/master/driver/driver_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo de Circuito de Comando
Lucas Seara Manoel e Diogo Tavares


![alt text](https://raw.githubusercontent.com/lsmanoel/PowerEletronics/master/driver/images/buck_circuit.PNG)

# Modelo 1: [TPS40052](http://www.ti.com/lit/ds/slus563c/slus563c.pdf)

![alt text](https://raw.githubusercontent.com/lsmanoel/PowerEletronics/master/driver/images/tps_40052_overview.png)

# Modelo 1: [TPS40056](http://www.ti.com/lit/ds/symlink/tps40056.pdf)
![alt text](https://raw.githubusercontent.com/lsmanoel/PowerEletronics/master/driver/images/tps_40056_overview.png)

# Modelo 2:[ MIC2103/4](http://ww1.microchip.com/downloads/en/DeviceDoc/20005899A.pdf)

![alt text](https://raw.githubusercontent.com/lsmanoel/PowerEletronics/master/driver/images/eletrical.PNG)

![alt text](https://raw.githubusercontent.com/lsmanoel/PowerEletronics/master/driver/images/aplication.PNG)